This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'MzQ2aUmy6S19GyH661zp'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key='+API_KEY)
print(r.json()['dataset_data']['column_names'])
print(r.json()['dataset_data']['data'][0])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2019-01-02', None, 70.85, 68.25, 69.85, None, None, None, None, None, None]


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
afx_x_dict = dict(r.json()['dataset_data'])
print(afx_x_dict.keys())
print('Start Date = ' + afx_x_dict['start_date'])
print('End Date = ' + afx_x_dict['end_date'])

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])
Start Date = 2000-06-07
End Date = 2019-01-02


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)? 
6. What was the average daily trading volume during this year? <font color='red'>(assumption: this year is 2017. Same applies to Q7)</font>
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# 1,2 - Collect data for 2017 into a Python dictionary

afx_2017_dict = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&order=asc&api_key='+API_KEY).json()['dataset_data']
print(afx_2017_dict.keys())
print('Start Date = ' + afx_2017_dict['start_date'])
print('End Date = ' + afx_2017_dict['end_date'])
print('Header = ' + str(afx_2017_dict['column_names']))
print('Data = ' + str(afx_2017_dict['data'][0]))

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])
Start Date = 2017-01-01
End Date = 2017-12-31
Header = ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
Data = ['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]


In [6]:
# 3 - Highest and lowest opening prices
opening_prices_2017 = []
dates = []
for daily_info in afx_2017_dict['data']:
    if daily_info[1] != None:
        opening_prices_2017.append(daily_info[1])
        dates.append(daily_info[0])

highest = max(opening_prices_2017)
lowest = min(opening_prices_2017)

print('Highest AFX price for 2017 = ' + str(highest) + ' on ' + dates[opening_prices_2017.index(highest)])
print('Lowest AFX price for 2017 = ' + str(lowest) + ' on ' + dates[opening_prices_2017.index(lowest)])

Highest AFX price for 2017 = 53.11 on 2017-12-14
Lowest AFX price for 2017 = 34.0 on 2017-01-24


In [7]:
# 4 - Largest change in any one day
Price_Change = []
dates = []
for daily_info in afx_2017_dict['data']:
    if (daily_info[2] != None) & (daily_info[3] != None):
        Price_Change.append(daily_info[2]-daily_info[3])
        dates.append(daily_info[0]) 
        
highest_change = max(Price_Change)
print('Highest AFX price change in 2017 = ' + str(round(highest_change,2)) + ' on ' + dates[Price_Change.index(highest_change)])

Highest AFX price change in 2017 = 2.81 on 2017-05-11


In [8]:
# 5 - Largest change in closing price between 2 days
closing_prices_2017 = []
dates = []
price_change = []

for daily_info in afx_2017_dict['data']:
    if daily_info[4] != None:
        closing_prices_2017.append(daily_info[4])
        dates.append(daily_info[0])

for i,close_price in enumerate(closing_prices_2017):
    if i==0:
        pass
    else:
        price_change.append(abs(closing_prices_2017[i]-closing_prices_2017[i-1]))

max_change = max(price_change)

print('Highest AFX closing price change in 2017 = ' + str(round(max_change,2)) + ' on ' + dates[price_change.index(max_change)+1])

Highest AFX closing price change in 2017 = 2.56 on 2017-08-09


In [9]:
# 6 - Average daily trading volume in 2017
import statistics as s

trading_volume_2017 = []
dates = []
for daily_info in afx_2017_dict['data']:
    if daily_info[6] != None:
        trading_volume_2017.append(daily_info[6])
        dates.append(daily_info[0])

avg_trd_volume =  sum(trading_volume_2017)/len(trading_volume_2017)

print('Computed average trading volume in 2017 = ' + str(round(avg_trd_volume,2)))
print('Derived from package average trading volume in 2017 = ' + str(round(s.mean(trading_volume_2017),2)))


Computed average trading volume in 2017 = 89124.34
Derived from package average trading volume in 2017 = 89124.34


In [10]:
# 7 - Median daily trading volume in 2017

def Median(lst):
    if len(lst)%2 == 0:
        med = (sorted(lst)[round(len(lst)/2)]+sorted(lst)[round((len(lst)/2)-1)])/2
    else:
        med = sorted(lst)[round((len(lst)/2)-0.5)]
    return med
            
print('Computed average trading volume in 2017 = ' + str(round(Median(trading_volume_2017),2)))
print('Derived from package average trading volume in 2017 = ' + str(round(s.median(trading_volume_2017),2)))


Computed average trading volume in 2017 = 76286.0
Derived from package average trading volume in 2017 = 76286.0
